In [12]:
import json
import pandas as pd
import numpy as np
from functools import reduce

In [13]:
def is_num(x):
    try:
        float(x)
        return True
    except:
        return False

In [14]:
f = open('./data_struct_info/data_struct.json')
data_struct = json.load(f)

In [15]:
# data_struct

In [16]:
f = open('./input_data/example.json')
jy_json = json.load(f)

In [17]:
ID = 60000
def append(arr, x):
    arr.append(x)
    return arr
def f(ac, x):
    zyh, type_res_obj = x
    def g(ac, x):
        t, res_arr = x
        reduce(lambda acc, y:append(acc, {
            **y,
            'type':t,
            'zyh':zyh
        }), res_arr,ac)
        return ac
    reduce(lambda acc,y:g(acc,y),type_res_obj.items(), ac )
    return ac
jy_list = reduce(f, jy_json.items(), [])
def f(x):
    offset, row = x
    return {
        **row, 
        'id':offset+ID
    }
jy_list = list(map(f, enumerate(jy_list) ))
# list(jy_json.items())[0]
jy_list[1]

{'时间': '20190409', '结果': 'A+', 'type': 'english', 'zyh': '0870', 'id': 60001}

In [18]:
ex_name = 'exam_result_list.xlsx'

def f(x):
    df_struct = data_struct[ex_name]
    row = [np.nan for i in df_struct['columns']]
    row[df_struct['column2pos']['exam_id']] = x['id']
    if is_num(x['结果']):
        row[data_struct[ex_name]['column2pos']['exam_result_number']] = x['结果']
    else:
        row[data_struct[ex_name]['column2pos']['exam_result_string']] = x['结果']
    return row
all_row = list(map(f, jy_list))
df = pd.DataFrame(all_row,columns=data_struct[ex_name]['columns'])
df.to_excel('./output_data/%s'%ex_name)

In [19]:
df

,exam_id,exam_result_number,exam_result_string
0,60000,97.0,NaN
1,60001,NaN,A+
2,60002,32.0,NaN
3,60003,NaN,G
